# Data Cleaning (Sean)

In [15]:
# Dependencies

import pandas as pd
from pathlib import Path
import numpy as np

from calendar import monthrange

## Chicago BikeShare CSV Data Cleaning and Merging

In [16]:
# Create a For loop to go through each monthly data set, clean and merge all Bike Ride Csvs

# Define an Empty Merged Data frame for Bike share Data

merged_rides = []

# Define ranges for Years of Documents and Months of Documents

years = np.arange(2020,2024,1)
months = np.arange(1,13,1)

# For loop that does through all months in a 3 year space

for y in years:
    
    for m in months:
        
        # Setting File Path for Bike share Data
        
        filepath = Path(f"../data/{y}{'{:02d}'.format(m)}-divvy-tripdata/{y}{'{:02d}'.format(m)}-divvy-tripdata.csv")
        
        # print(filepath)
        
        try:
            
            # read csv
            
            trip_data = pd.read_csv(filepath)
            
            # limit to chicago/ weather applicable data
            
            trip_data['start_lat'] = round(trip_data['start_lat'], 1)
            trip_data['start_lng'] = round(trip_data['start_lng'], 1)
            
            trip_data_chicago = trip_data.loc[(trip_data['start_lat'] == 41.9) 
                                & (trip_data['start_lng'] == -87.6), :]
            
            
            # remove irrelevant data columns
            
            trip_sorted_data = trip_data_chicago[['ride_id', 'started_at', 'ended_at', 'start_station_name', 
                                                'member_casual', 'rideable_type']].copy()
            
            
            # translate started at to Start Date and Start Hour
            
            trip_sorted_data['started_at'] = trip_sorted_data['started_at'].astype('datetime64[s]')
            trip_sorted_data['start_day'] = trip_sorted_data['started_at'].dt.date
            trip_sorted_data['start_hour'] = trip_sorted_data['started_at'].dt.hour
            trip_sorted_data['start_month'] = trip_sorted_data['started_at'].dt.month
            trip_sorted_data['start_year'] = trip_sorted_data['started_at'].dt.year
            
            # add new TRIP LENGTH column
            
            trip_sorted_data['started_at'] = pd.to_datetime(trip_sorted_data['started_at'])
            trip_sorted_data['ended_at'] = pd.to_datetime(trip_sorted_data['ended_at'])
            
            trip_sorted_data['trip_length'] = trip_sorted_data['ended_at'] - trip_sorted_data['started_at']
            
            # remove bad data & remove/ reorder Columns
            
            trip_data_clean = trip_sorted_data.loc[trip_sorted_data['trip_length'] > "P0DT0H0M0S", :]
            trip_data_clean = trip_data_clean.dropna(how = 'any')
            
            trip_data_clean = trip_data_clean[['ride_id', 'started_at', 'start_day', 'start_hour', 'start_month', 'start_year', 
                                               'trip_length', 'start_station_name', 'member_casual', 'rideable_type']].copy()
            
            # sort by started_at and fix index
            
            trip_data_clean = trip_data_clean.sort_values(['start_day', 'start_hour']).set_index('ride_id').reset_index()
            
            # display(trip_data_clean.count())
            
            merged_rides.append(trip_data_clean)
            
        # Skip Non-Existing files
            
        except FileNotFoundError:
            
            pass
            
            # check which files are skipped
            
            # print(f"an Exception occured at - {y}{'{:02d}'.format(m)}")



combined_bike_df = pd.concat(merged_rides, ignore_index=True)

combined_bike_df

,ride_id,started_at,start_day,start_hour,start_month,start_year,trip_length,start_station_name,member_casual,rideable_type
0,782CEA3C6968D2A6,2020-04-01 00:13:41,2020-04-01,0,4,2020,0 days 00:05:28,Kingsbury St & Erie St,member,docked_bike
1,07F785C9DDA3404C,2020-04-01 00:11:18,2020-04-01,0,4,2020,0 days 00:00:33,Wabash Ave & 9th St,member,docked_bike
2,1FD159E93F7BAFA1,2020-04-01 00:02:35,2020-04-01,0,4,2020,0 days 00:08:10,Wabash Ave & 16th St,member,docked_bike
3,091D47E4F0FC5022,2020-04-01 00:06:44,2020-04-01,0,4,2020,0 days 00:07:17,Mies van der Rohe Way & Chicago Ave,member,docked_bike
4,643593E85E46A45C,2020-04-01 00:13:36,2020-04-01,0,4,2020,0 days 00:05:23,Kingsbury St & Erie St,member,docked_bike
...,...,...,...,...,...,...,...,...,...,...
9813892,6D855DB843848DB3,2023-11-30 23:16:11,2023-11-30,23,11,2023,0 days 00:10:14,Canal St & Adams St,member,classic_bike
9813893,447027EB102601BE,2023-11-30 23:02:56,2023-11-30,23,11,2023,0 days 00:21:13,Lincoln Ave & Fullerton Ave,casual,electric_bike
9813894,993257B9E439A2DD,2023-11-30 23:53:54,2023-11-30,23,11,2023,0 days 00:09:21,Wentworth Ave & Cermak Rd*,member,classic_bike
9813895,9B518D5122FD7D72,2023-11-30 23:53:34,2023-11-30,23,11,2023,0 days 00:09:43,Wentworth Ave & Cermak Rd*,member,classic_bike


## Chicago Weather CSV Data Cleaning and Merging

In [17]:

# Start Weather Data Data Cleaning

# Setting File Path for Csv

weather_filepath = Path('../data/chicago_weather_data_2020_2023_Celcius.csv')

# Read Csv data

weather_data = pd.read_csv(weather_filepath)

# Convert Started at to DT

weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'], format='%Y-%m-%d %H:%M:%S %z UTC', utc=True)

# Convert UTC to Chicago time by designating a time zone

weather_data['dt_iso_chicago'] = weather_data['dt_iso'].dt.tz_convert('America/Chicago')

# translate started at into start day and start hour

weather_data['start_day'] = weather_data['dt_iso_chicago'].dt.date
weather_data['start_hour'] = weather_data['dt_iso_chicago'].dt.hour

# remove irrelevant data columns and duplicated data

weather_data_sorted = weather_data[['start_day', 'start_hour', 'temp', 'dew_point', 'feels_like', 
                                'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'wind_gust',
                                'clouds_all', 'weather_id', 'weather_main', 'weather_description']].copy()

weather_data_sorted = weather_data_sorted.drop_duplicates(subset=['start_day', 'start_hour'])

weather_data_sorted



,start_day,start_hour,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,clouds_all,weather_id,weather_main,weather_description
0,2019-12-31,18,-2.08,-5.63,-9.08,-2.31,-1.56,1010,74,12.90,260,0.00,95,804,Clouds,overcast clouds
1,2019-12-31,19,-1.99,-5.23,-8.99,-2.23,-1.54,1010,76,10.30,270,0.00,75,803,Clouds,broken clouds
2,2019-12-31,20,-1.87,-4.96,-8.87,-2.23,-1.36,1011,77,9.80,270,0.00,100,804,Clouds,overcast clouds
3,2019-12-31,21,-1.90,-4.55,-8.90,-2.79,-1.14,1011,80,9.30,270,0.00,40,802,Clouds,scattered clouds
4,2019-12-31,22,-2.18,-4.67,-9.18,-2.78,-1.36,1011,81,8.80,260,0.00,75,803,Clouds,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37232,2023-12-31,13,0.72,-1.19,-5.23,0.00,2.22,1014,86,7.60,290,11.62,100,600,Snow,light snow
37234,2023-12-31,14,0.63,-1.70,-5.17,0.00,2.02,1014,83,7.15,291,10.28,100,600,Snow,light snow
37236,2023-12-31,15,0.88,-1.19,-4.85,0.53,2.02,1015,85,7.15,289,9.39,100,600,Snow,light snow
37238,2023-12-31,16,0.77,-1.15,-4.38,-0.03,2.22,1016,86,5.81,287,7.15,100,600,Snow,light snow


## Merge the Data Frames

In [18]:
# Merge the Datasets

result = pd.merge(combined_bike_df, weather_data_sorted, on=['start_day', 'start_hour'], how='inner')

# return1 = result.loc[result['start_hour'] == 1, :]

result

,ride_id,started_at,start_day,start_hour,start_month,start_year,trip_length,start_station_name,member_casual,rideable_type,...,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,clouds_all,weather_id,weather_main,weather_description
0,782CEA3C6968D2A6,2020-04-01 00:13:41,2020-04-01,0,4,2020,0 days 00:05:28,Kingsbury St & Erie St,member,docked_bike,...,4.02,1018,86,6.70,350,0.0,100,804,Clouds,overcast clouds
1,07F785C9DDA3404C,2020-04-01 00:11:18,2020-04-01,0,4,2020,0 days 00:00:33,Wabash Ave & 9th St,member,docked_bike,...,4.02,1018,86,6.70,350,0.0,100,804,Clouds,overcast clouds
2,1FD159E93F7BAFA1,2020-04-01 00:02:35,2020-04-01,0,4,2020,0 days 00:08:10,Wabash Ave & 16th St,member,docked_bike,...,4.02,1018,86,6.70,350,0.0,100,804,Clouds,overcast clouds
3,091D47E4F0FC5022,2020-04-01 00:06:44,2020-04-01,0,4,2020,0 days 00:07:17,Mies van der Rohe Way & Chicago Ave,member,docked_bike,...,4.02,1018,86,6.70,350,0.0,100,804,Clouds,overcast clouds
4,643593E85E46A45C,2020-04-01 00:13:36,2020-04-01,0,4,2020,0 days 00:05:23,Kingsbury St & Erie St,member,docked_bike,...,4.02,1018,86,6.70,350,0.0,100,804,Clouds,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9813892,6D855DB843848DB3,2023-11-30 23:16:11,2023-11-30,23,11,2023,0 days 00:10:14,Canal St & Adams St,member,classic_bike,...,7.81,1012,92,4.92,183,7.6,100,500,Rain,light rain
9813893,447027EB102601BE,2023-11-30 23:02:56,2023-11-30,23,11,2023,0 days 00:21:13,Lincoln Ave & Fullerton Ave,casual,electric_bike,...,7.81,1012,92,4.92,183,7.6,100,500,Rain,light rain
9813894,993257B9E439A2DD,2023-11-30 23:53:54,2023-11-30,23,11,2023,0 days 00:09:21,Wentworth Ave & Cermak Rd*,member,classic_bike,...,7.81,1012,92,4.92,183,7.6,100,500,Rain,light rain
9813895,9B518D5122FD7D72,2023-11-30 23:53:34,2023-11-30,23,11,2023,0 days 00:09:43,Wentworth Ave & Cermak Rd*,member,classic_bike,...,7.81,1012,92,4.92,183,7.6,100,500,Rain,light rain


## Export Merged CSV

In [19]:
# save file

export_merged_filepath = Path('../output/merged_weather_bike_data.csv')

result.to_csv(export_merged_filepath, index = False)